The wrapper of the JSON API can be pip installed from my fork, required for geojson search

!pip install git+https://github.com/rbavery/usgs

In [1]:
import json
import getpass
import requests
import wget
import sys
from usgs import api
import pandas as pd

# get api key

In [223]:
#Form JSON Object to store credentials to generate API Key.
username='rbavery'
password = getpass.getpass()

login = r'https://earthexplorer.usgs.gov/inventory/json/v/1.4.0/login'

creds={"username": username,
"password": password,
"authType": "",
"catalogId": "EE"}

json_data = {'jsonRequest' : json.dumps(creds)}

r = requests.post(login, data = json_data)
token = r.json()['data'].replace("'",'"') # json needs double quotes

if token == None:
    print("API Key Failed to Generate")

else:
    print("Your API Key Generated Successfully")
    print(token)
    
def check_item_basket(token):
    item_basket_json = {
        "apiKey" : token
    }

    api_string = r'https://earthexplorer.usgs.gov/inventory/json/v/1.4.0/'
    get_bd =api_string+"itembasket"

    json_data = {'jsonRequest' : json.dumps(item_basket_json)}

    r = requests.post(get_bd, data = json_data)
    return r.json()

def submit_bulk_order(token):
    request_json = {
        "apiKey" : token
    }

    api_string = r'https://earthexplorer.usgs.gov/inventory/json/v/1.4.0/'
    get_bd =api_string+"submitbulkdownloadorder"

    json_data = {'jsonRequest' : json.dumps(request_json)}

    r = requests.post(get_bd, data = json_data)
    return r.json()

#If Key fails to generate please see documentmentation to decode error https://earthexplorer.usgs.gov/inventory/documentation/errors

········
Your API Key Generated Successfully
78492e3e5a984a8aac83846535278cf7


# make a geojson feature

In [3]:
geojson_aoi = feature_coll = {
"type": "FeatureCollection",
"features": [
{
"type": "Feature",
"properties": {},
"geometry": {
"type": "Polygon",
"coordinates": [
[
[
-104.1064453125,
43.13306116240612
],
[
-104.12841796875,
41.04621681452063
],
[
-102.19482421875,
41.0130657870063
],
[
-102.15087890624999,
40.027614437486655
],
[
-95.07568359375,
40.01078714046552
],
[
-95.625,
40.730608477796636
],
[
-96.13037109375,
42.17968819665961
],
[
-97.27294921875,
43.02071359427862],
[
-98.02001953125,
42.90816007196054
],
[
-98.54736328125,
43.08493742707592
],
[
-104.1064453125,
43.13306116240612
]
]
]
}
}
]
}

# get dataset fields

In [5]:
def make_date_strings(start_date, end_date):

    return [i.date() for i in pd.date_range(start_date, end_date).tolist() if i.month in [6, 7, 8, 9]]

In [6]:
import numpy as np
start_dates = [str(i) + "-06-01" for i in np.arange(2002, 2020)]
end_dates = [str(i) + "-09-30" for i in np.arange(2002, 2020)]

In [7]:
all_results = []
for start_date, end_date in zip(start_dates, end_dates):
    results = api.search(dataset="ARD_TILE", node="EE", start_date=start_date, end_date=end_date, ll={"longitude":-104.1064453125, "latitude":39.825413103424786}, ur={"longitude":-95.130615234375, "latitude":43.100982876188546}, api_key=token)
    all_results.extend(results['data']['results'])
    print(start_date+" done")

2002-06-01 done
2003-06-01 done
2004-06-01 done
2005-06-01 done
2006-06-01 done
2007-06-01 done
2008-06-01 done
2009-06-01 done
2010-06-01 done
2011-06-01 done
2012-06-01 done
2013-06-01 done
2014-06-01 done
2015-06-01 done
2016-06-01 done
2017-06-01 done
2018-06-01 done
2019-06-01 done


In [18]:
filtered_results = []
for i in all_results:
    if "landsat_etm_c1" in i['browseUrl']:
        pass
    else:
        if i['cloudCover'] == None:
            filtered_results.append(i)
            
        elif i['cloudCover'] <= 10:
            filtered_results.append(i)
        
        else:
            pass


In [183]:
entity_ids = []
for i in filtered_results:
    entity_ids.append("{}".format(i['entityId']))

last search yielded up t o this: https://earthexplorer.usgs.gov/download/14320/LC08_CU_012006_20190909_C01_V01/TOA/EE

In [37]:
download_opt_json = {
    "datasetName" : "ARD_TILE",
    "apiKey" : token,
    "entityIds": entity_ids
}

api_string = r'https://earthexplorer.usgs.gov/inventory/json/v/1.4.0/'
get_bd =api_string+"downloadoptions"

json_data = {'jsonRequest' : json.dumps(download_opt_json)}

r = requests.post(get_bd, data = json_data)

In [39]:
r.json()['data'][0]

{'downloadOptions': [{'available': True,
   'downloadCode': 'SR',
   'filesize': 6430720,
   'productCode': None,
   'productName': 'Surface Reflectance',
   'url': 'https://earthexplorer.usgs.gov/download/14320/LT05_CU_013007_20020930_C01_V01/SR/INVSVC/',
   'storageLocation': 'SILO'},
  {'available': True,
   'downloadCode': 'ST',
   'filesize': 4505600,
   'productCode': None,
   'productName': 'Provisional Surface Temperature ',
   'url': 'https://earthexplorer.usgs.gov/download/14320/LT05_CU_013007_20020930_C01_V01/ST/INVSVC/',
   'storageLocation': 'SILO'},
  {'available': True,
   'downloadCode': 'TOA',
   'filesize': 6246400,
   'productCode': None,
   'productName': 'Top of Atmosphere',
   'url': 'https://earthexplorer.usgs.gov/download/14320/LT05_CU_013007_20020930_C01_V01/TOA/INVSVC/',
   'storageLocation': 'SILO'},
  {'available': True,
   'downloadCode': 'BT',
   'filesize': 890880,
   'productCode': None,
   'productName': 'Brightness Temperature',
   'url': 'https://eart

In [76]:
bulk_order_request = {
    "datasetName": "ARD_TILE",
    "apiKey": token,
    "entityIds": entity_ids
}


get_bd =api_string+"getbulkdownloadproducts"

json_data = {'jsonRequest' : json.dumps(bulk_order_request)}

r = requests.post(get_bd, data = json_data)

bd_products = r.json()

In [16]:
download_json = {
    "datasetName" : "ARD_TILE",
    "products" : ["Surface Reflectance",
                  "Provisional Surface Temperature",
                  "Brightness Temperature",
                  "Quality Assessment",
                  "Full Resolution Browse",
                  "Metadata"],
    "apiKey" : token,
    "entityIds": entity_ids
}


In [17]:
lst_of_update_requests = [ {
    "apiKey": token,
    "datasetName": "ARD_TILE",
    "entityId": product_set["entityId"],
    "downloadCodes": [i['downloadCode'] for i in product_set["products"]]
} for product_set in bd_products['data'] ]

In [44]:
req_chunks = [list(i) for i in [*np.array_split(lst_of_update_requests, len(lst_of_update_requests)/500)]]

In [46]:
# regular download doesn't work, ard needs to be bulk ordered
get_bd =api_string+"updatebulkdownloadscene"

for chunk in req_chunks:
    for request in chunk:
        json_data = {'jsonRequest' : json.dumps(request)}

        r = requests.post(get_bd, data = json_data)
    submit_bulk_order(token)

These entity ids don't work, they were returned from a call to the api search endpoint

In [191]:
chunk[0:7]

['LT05_CU_013007_20100902_C01_V01',
 'LT05_CU_013008_20100902_C01_V01',
 'LT05_CU_013009_20100902_C01_V01',
 'LT05_CU_012006_20100831_C01_V01',
 'LT05_CU_012007_20100831_C01_V01',
 'LT05_CU_015009_20100830_C01_V01',
 'LT05_CU_016007_20100830_C01_V01']

These do, just because they have been interpreted by python rather than returned from a call to the api search endpoint

In [195]:
chunk2 = ['LT05_CU_013007_20100902_C01_V01',
 'LT05_CU_013008_20100902_C01_V01',
 'LT05_CU_013009_20100902_C01_V01',
 'LT05_CU_012006_20100831_C01_V01',
 'LT05_CU_012007_20100831_C01_V01',
 'LT05_CU_015009_20100830_C01_V01',
 'LT05_CU_016007_20100830_C01_V01']
chunk2

['LT05_CU_013007_20100902_C01_V01',
 'LT05_CU_013008_20100902_C01_V01',
 'LT05_CU_013009_20100902_C01_V01',
 'LT05_CU_012006_20100831_C01_V01',
 'LT05_CU_012007_20100831_C01_V01',
 'LT05_CU_015009_20100830_C01_V01',
 'LT05_CU_016007_20100830_C01_V01']

This was failing with a JSON decode error until I changed the chunk size to 100.

In [ ]:
chunk_size = 100

In [221]:
urlList[1303:1305]

['https://dds.cr.usgs.gov/ltaauth/hsm/lsat2/collection01/tm/ARD_Tile/2002/CU/014/008/LT05_CU_014008_20020728_20190111_C01_V01_ST.tar?id=qcpt4brqkgbk8at57tfn0hv5g2&iid=LT05_CU_014008_20020728_C01_V01&did=525069389&ver=production',
 'https://dds.cr.usgs.gov/ltaauth/hsm/lsat2/collection01/tm/ARD_Tile/2002/CU/014/008/LT05_CU_014008_20020728_20190111_C01_V01_BT.tar?id=qcpt4brqkgbk8at57tfn0hv5g2&iid=LT05_CU_014008_20020728_C01_V01&did=525069390&ver=production']

In [229]:
len(urlList)

40188

In [215]:
entity_chunks = [list(i) for i in [*np.array_split(entity_ids, len(entity_ids)/chunk_size)]]
download_lists = []
downloadURL="https://earthexplorer.usgs.gov/inventory/json/v/1.4.1/download"

for chunk in entity_chunks:
    
    orderParms ={'datasetName':"ARD_TILE",
          'products':['SR',
                      'ST',
                      'BT',
                      'QA',
                      'FRB',
                      'METADATA'],
          'entityIds': chunk,
          "apiKey": (token)

          }

    download= requests.get(downloadURL, params={'jsonRequest':json.dumps(orderParms)})

    print(download)
    download_list = download.json()['data']

    download_lists.append(download_list)
    
urlList = []
for lst in download_lists:
    for i in lst:
        urlList.append(i['url'])

#Users will need to specify a download directory on their machine.
outDir="/home/rave/forge_scratch/ARD-all-nebraska-2002-2019/"
for URL in urlList:
    wget.download(URL, out=outDir, bar=None)
print("The Following Scenes have downloaded:")
print(entity_chunks)
print("Download Complete")

<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200

ContentTooShortError: <urlopen error retrieval incomplete: got only 0 out of 52736000 bytes>

In [230]:
#Users will need to specify a download directory on their machine.
outDir="/home/rave/forge_scratch/ARD-all-nebraska-2002-2019/"
for URL in urlList[1304:]:
    wget.download(URL, out=outDir, bar=None)
print("The Following Scenes have downloaded:")
print(entity_chunks)
print("Download Complete")

ContentTooShortError: <urlopen error retrieval incomplete: got only 0 out of 11663360 bytes>

In [214]:
download

<Response [200]>

In [77]:
import csv 
with open('/home/rave/forge_scratch/dowload_urls.csv', 'r') as f: 
    reader = csv.reader(f) 
    download_list = list(reader)[0]

In [81]:
outDir="/home/rave/forge_scratch/ARD-all-nebraska-2002-2019/"
wget.download(download_list[0], out=outDir)

'/home/rave/forge_scratch/ARD-all-nebraska-2002-2019//EE'

In [ ]:
outDir="/home/rave/forge_scratch/ARD-all-nebraska-2002-2019/"
for URL in urlList:
    wget.download(URL, out=outDir, bar=None)
print("The Following Scenes have downloaded:")
print(order)
print("Download Complete")

All of the above gets you the bulk downloads, only to have the links require a browser to download to login, os headless download doesn't work. need to use bulk-download app.

Creates download list, schema for downloa durl found through the web browwser with chrome inspect tool.

In [95]:
download_list = []
for i in bd_products['data']:
    for j in i['products']:
        download_list.append(j['url'].replace("INVSVC/", "")+"EE")

In [97]:
import csv
with open("dowload_urls.csv", 'w', newline='') as myfile:
     wr = csv.writer(myfile, quoting=csv.QUOTE_ALL)
     wr.writerow(download_list)

In [ ]:
api.dataset_fields("ARD_TILE", node="EE", api_key=token)

# There should be more results returned from Nebraska (the geojson used to search) throughout the year, I've confirmed this by downloading ARD manually.

# scratch code below

In [ ]:
all_results_good_tiles = []
for i in all_results:
    horizontal_grid = i['browseUrl'].split('/')[8]
    vertical_grid = i['browseUrl'].split('/')[9]
    if horizontal_grid in ["011","012","013","014","015","016","017"] and vertical_grid in ["006","007","008","009","010"]:
        all_results_good_tiles.append(i)

In [ ]:
where_and_dict = {
     21787 : ["011","012","013","014","015","016","017"], # tile grid horizontal
     21788 : ["006","007","008","009", "010"], # tile grid vertical
     21791 : '10', # cloud cover
     21792 : ['OLI_TIRS', 'TM']
}

additionalCriteria = {
    "filterType": "and",
    "childFilters": [
        {
            "filterType": "value",
            "fieldId": field_id,
            "value": value,
            "operand": "="
        }
        for field_id, value in iter(where.items())
    ]
}

In [96]:
searchUrl='https://earthexplorer.usgs.gov/inventory/json/v/1.4.1/search'
dataset_search = {"apiKey": (token), "datasetName": "ARD_TILE"}
dataset_request =requests.get(searchUrl, params={"jsonRequest":json.dumps(dataset_search)})
d_resultslist = dataset_request.json()['data']['results']
order = []
for res in d_resultslist:
    order.append(res['entityId'])

#Check to see if query resulted in scenes being returned.
if len(order) == 0:
    sys.exit("No Scenes Matching Criteria Were Found, Program will terminate")
else:
    print("Your Scene List Contains:")
    print(order)

Your Scene List Contains:
['LT04_CU_028004_19821111_C01_V01', 'LT04_CU_028005_19821111_C01_V01', 'LT04_CU_028007_19821111_C01_V01', 'LT04_CU_028008_19821111_C01_V01', 'LT04_CU_028009_19821111_C01_V01', 'LT04_CU_028010_19821111_C01_V01', 'LT04_CU_028011_19821111_C01_V01', 'LT04_CU_028012_19821111_C01_V01', 'LT04_CU_029004_19821111_C01_V01', 'LT04_CU_029005_19821111_C01_V01']


In [22]:
import pandas as pd

In [43]:
def read_dataset_json(fname):
    return pd.DataFrame(pd.read_json(fname).data.apply(lambda x: x['datasetName']))

In [91]:
download_json_r

{'datasetName': 'ARD_TILE',
 'products': ['Surface Reflectance',
  'Provisional Surface Temperature',
  'Brightness Temperature',
  'Quality Assessment',
  'Full Resolution Browse',
  'Metadata'],
 'apiKey': '48abdaf089aa476ba6b98377132eb812',
 'entityIds': ['LT05_CU_012008_20020921_C01_V01',
  'LT05_CU_012009_20020921_C01_V01',
  'LT05_CU_013008_20020921_C01_V01',
  'LT05_CU_013009_20020921_C01_V01',
  'LT05_CU_014008_20020921_C01_V01']}

In [173]:
chunk

['LT05_CU_013007_20020930_C01_V01',
 'LT05_CU_013008_20020930_C01_V01',
 'LT05_CU_013009_20020930_C01_V01',
 'LT05_CU_014007_20020930_C01_V01',
 'LT05_CU_014008_20020930_C01_V01',
 'LT05_CU_014009_20020930_C01_V01',
 'LT05_CU_015007_20020930_C01_V01',
 'LT05_CU_015008_20020930_C01_V01',
 'LT05_CU_012006_20020928_C01_V01',
 'LT05_CU_012007_20020928_C01_V01',
 'LT05_CU_012008_20020928_C01_V01',
 'LT05_CU_012009_20020928_C01_V01',
 'LT05_CU_013007_20020928_C01_V01',
 'LT05_CU_013008_20020928_C01_V01',
 'LT05_CU_013009_20020928_C01_V01',
 'LT05_CU_017009_20020927_C01_V01',
 'LT05_CU_012006_20020926_C01_V01',
 'LT05_CU_012007_20020926_C01_V01',
 'LT05_CU_015009_20020925_C01_V01',
 'LT05_CU_016007_20020925_C01_V01',
 'LT05_CU_016008_20020925_C01_V01',
 'LT05_CU_016009_20020925_C01_V01',
 'LT05_CU_017007_20020925_C01_V01',
 'LT05_CU_017008_20020925_C01_V01',
 'LT05_CU_017009_20020925_C01_V01',
 'LT05_CU_014007_20020923_C01_V01',
 'LT05_CU_014008_20020923_C01_V01',
 'LT05_CU_014009_20020923_C0

In [176]:
order2 = ['LT05_CU_013007_20020930_C01_V01']

In [202]:
chunk[0]

'LC08_CU_012007_20180713_C01_V01'

In [206]:
#Create routine to generate download URLs for desired product for enity ids from search query. Please consult Get Bulk Download
#product endpoint for available products per dataset. Here Level-1 Landsat Geotiff will be downloaded. The product is downloaded
#using wget.

orderParms ={'datasetName':"ARD_TILE",
      'products':['SR',
                  'ST',
                  'BT',
                  'QA',
                  'FRB',
                  'METADATA'],
      'entityIds': chunk[0:11],
      "apiKey": (token)

      }

downloadURL="https://earthexplorer.usgs.gov/inventory/json/v/1.4.1/download"
download= requests.get(downloadURL, params={'jsonRequest':json.dumps(orderParms)})

urlList = []
download_list = download.json()['data']
for item in download_list:
    urlList.append(item['url'])

#Users will need to specify a download directory on their machine.
outDir="/home/rave/forge_scratch/ARD-all-nebraska-2002-2019/"
for URL in urlList:
    wget.download(URL, out=outDir, bar=None)
print("The Following Scenes have downloaded:")
print(order2)
print("Download Complete")

KeyboardInterrupt: 

In [172]:
download.json()

{'errorCode': None,
 'error': '',
 'data': [{'entityId': 'LT04_CU_028004_19821111_C01_V01',
   'product': 'SR',
   'url': 'https://dds.cr.usgs.gov/ltaauth/hsm/lsat2/collection01/tm/ARD_Tile/1982/CU/028/004/LT04_CU_028004_19821111_20190206_C01_V01_SR.tar?id=0aggsmhnsg773frh48had8jkb3&iid=LT04_CU_028004_19821111_C01_V01&did=525056820&ver=production'},
  {'entityId': 'LT04_CU_028004_19821111_C01_V01',
   'product': 'ST',
   'url': 'https://dds.cr.usgs.gov/ltaauth/hsm/lsat2/collection01/tm/ARD_Tile/1982/CU/028/004/LT04_CU_028004_19821111_20190206_C01_V01_ST.tar?id=0aggsmhnsg773frh48had8jkb3&iid=LT04_CU_028004_19821111_C01_V01&did=525056821&ver=production'},
  {'entityId': 'LT04_CU_028004_19821111_C01_V01',
   'product': 'BT',
   'url': 'https://dds.cr.usgs.gov/ltaauth/hsm/lsat2/collection01/tm/ARD_Tile/1982/CU/028/004/LT04_CU_028004_19821111_20190206_C01_V01_BT.tar?id=0aggsmhnsg773frh48had8jkb3&iid=LT04_CU_028004_19821111_C01_V01&did=525056822&ver=production'},
  {'entityId': 'LT04_CU_0280

In [106]:
download.json()

{'errorCode': 'INVALID_INPUT',
 'error': 'Invalid product used for this dataset',
 'data': None,
 'api_version': '1.4.1',
 'access_level': 'approved',
 'catalog_id': 'EE',
 'executionTime': 0.2992680072784424}

In [129]:
bulk_order_request = {
    "datasetName": "ARD_TILE",
    "apiKey": token,
    "entityIds": order
}


get_bd =api_string+"getbulkdownloadproducts"

json_data = {'jsonRequest' : json.dumps(bulk_order_request)}

r = requests.post(get_bd, data = json_data)

bd_products = r.json()

In [130]:
bd_products

{'errorCode': None,
 'error': '',
 'data': [{'entityId': 'LT04_CU_028004_19821111_C01_V01',
   'products': [{'available': True,
     'downloadCode': 'SR',
     'filesize': 92631040,
     'productCode': None,
     'productName': 'Surface Reflectance',
     'url': 'https://earthexplorer.usgs.gov/download/14320/LT04_CU_028004_19821111_C01_V01/SR/INVSVC/',
     'storageLocation': None},
    {'available': True,
     'downloadCode': 'ST',
     'filesize': 42844160,
     'productCode': None,
     'productName': 'Provisional Surface Temperature ',
     'url': 'https://earthexplorer.usgs.gov/download/14320/LT04_CU_028004_19821111_C01_V01/ST/INVSVC/',
     'storageLocation': None},
    {'available': True,
     'downloadCode': 'TOA',
     'filesize': 72878080,
     'productCode': None,
     'productName': 'Top of Atmosphere',
     'url': 'https://earthexplorer.usgs.gov/download/14320/LT04_CU_028004_19821111_C01_V01/TOA/INVSVC/',
     'storageLocation': None},
    {'available': True,
     'downloa